In [ ]:
import pandas as pd
import os
from langchain_openai import ChatOpenAI
import ast
import requests
import time
import numpy as np
from pathlib import Path
import re
import requests
from tqdm import tqdm

In [ ]:
# Insert your own openai api key 
os.environ["OPENAI_API_KEY"] = "your_api_key"

output_file = "yelp_query_p.csv" # where to save data

In [ ]:
template = """ You are an expert in spatial keyword searching and I am now trying to perform spatial keyword searching using a large language model. In order to get a test set, I need you to help me write query questions based on the information I provide. In particular, I am asking to think of some questions that are difficult to answer with traditional keyword matching techniques, but are conveniently solved by the semantic techniques of the large language model, such as Find Japanese restaurants in Center City that offer a variety of sushi options, a query in which Japanese restaurants and sushi in this query can be easily solved by keyword matching, but a variety of options may require semantic techniques, and I need questions that are best solved only by semantic techniques. Also, please don't mention any location information in the query!

information: Pep Boys is located at Lafayette Road and primarily serves the category of Automotive, Tires, Oil Change Stations, Auto Parts & Supplies, Auto Repair. It is open for business at these hours: ['Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', 'Wednesday': '8:0-19:0', 'Thursday': '8:0-19:0', 'Friday': '8:0-19:0', 'Saturday': '8:0-19:0', 'Sunday': '9:0-17:0']. Customers often highlight: 'The reviews consistently praise the staff for being friendly, knowledgeable, and helpful, creating a positive and welcoming atmosphere for customers.' which suggests it's a popular choice among visitor
question: My car needs repairs. Which service center is the most reliable?

information: Biscuits Cafe is located at Broad Ripple Avenue, Broad Ripple and primarily serves the category of Restaurants, Mexican, Breakfast & Brunch. It is open for business at these hours: ['Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', 'Wednesday': '7:0-15:0', 'Thursday': '7:0-15:0', 'Friday': '7:0-15:0', 'Saturday': '7:0-15:0', 'Sunday': '7:0-15:0']. Customers often highlight: 'Customers rave about the fast-paced and eventful experience at this breakfast spot in Indy, praising dishes like spicy biscuits and gravy, tamales, steak tacos, chilaquiles, breakfast chimichanga, and Mexican omelet. The use of Ribeye for the meat and the authentic Mexican take on favorites are highlighted. The restaurant is known for its friendly staff, quick service, and generous portions at great prices. However, there are mentions of high prices, occasional closures, and mixed reviews on items like biscuits and gravy. Overall, it's a popular choice for those seeking a vibrant breakfast or brunch spot with a Mexican flair.' which suggests it's a popular choice among visitors.
question: Is there a place where I can try authentic Mexican food?

now it is your turn.
information: {input}
question: 
"""

# question = """ now it is your turn.
# information: {input}
# question: 
# """

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()


# system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template(template)
prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
output_parser = StrOutputParser()

llm = ChatOpenAI(temperature=1, model="o1-mini")
chain = prompt_template | llm | output_parser

In [ ]:
df = pd.read_csv('yelp_final_full_p.csv')
df.columns

In [ ]:
sampled_df = df[['business_id','name','latitude','longitude','categories','postcode','new_neighborhood', 'county','description']].sample(n=200, random_state=42)
sampled_df.columns

In [ ]:
def generate_query(infor):
    response = chain.invoke({"input": infor})
    return response

print("Start...")
sampled_df['question'] = sampled_df['description'].apply(generate_query)

sampled_df.to_csv(output_file, index=False)
print(f"Finish, has been saved to {output_file}。")